In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from bsd_dataset import get_dataset, regions, DatasetRequest

In [3]:
if torch.cuda.is_available():
    print(torch.cuda.device_count())

1


In [4]:
input_datasets = [
    DatasetRequest(
        dataset='projections-cmip6',
        model='gfdl_esm4',
        variable='precipitation',
    ),
    DatasetRequest(
       dataset='gmted2010',
       resolution=0.25
    )
]

target_dataset = DatasetRequest(dataset='chirps', resolution=0.25)

In [26]:
dataset = get_dataset(
    input_datasets,
    target_dataset,
    train_region=regions.California,
    val_region=regions.California,
    test_region=regions.California,
    train_dates=('1990-01-01', '1990-12-31'),
    val_dates=('1991-01-01', '1991-12-31'),
    test_dates=('1992-01-01', '1992-12-31'),
    download=False,  # CHANGE ME (as needed)
    extract=True,   # CHANGE ME (as needed)
    device='cuda:0'
)

OSError: no files to open

In [12]:
train_dataset = dataset.get_split('train')
val_dataset = dataset.get_split('val')
test_dataset = dataset.get_split('test')

In [17]:
x, y, mask = train_dataset[0]

print(f'INPUT SHAPE: {x.shape} ({x.device})')
print(f'TARGET SHAPE: {y.shape} ({x.device})')
print(f'MASK SHAPE: {mask.shape} ({x.device})')

INPUT SHAPE: torch.Size([3, 40, 40]) (cuda:0)
TARGET SHAPE: torch.Size([40, 40]) (cuda:0)
MASK SHAPE: torch.Size([40, 40]) (cuda:0)
